In [1]:
import os
import requests
import numpy as np
import pandas as pd

from sklearn.decomposition import PCA, FastICA

import statsmodels.tsa.api as tsa
from scipy.stats import probplot, moment

import matplotlib.pyplot as plt

from api_keys import fmp_api_key as api_key
from API import API

api = API(api_key)
X = api.getSamples("TSLA")

display(X)
print(X.dtypes)
print(len(X))

,symbol,date,currentRatio,quickRatio,grossProfitMargin,operatingProfitMargin,returnOnAssets,returnOnEquity,returnOnCapitalEmployed,debtEquityRatio,...,growthCostOfRevenue,growthGrossProfit,growthGrossProfitRatio,growthOperatingExpenses,growthEBITDA,growthOperatingIncome,growthNetIncome,growthEPS,marketCapitalization,enterpriseValue
0,TSLA,2016-12-31,1.074273,0.667986,0.228461,-0.095332,-0.029779,-0.142000,-0.044328,3.603263,...,0.729652,0.731729,0.000927,0.381960,-2.195635,-0.068779,-0.240529,-0.324675,3.614385e+10,3.987907e+10
1,TSLA,2017-12-31,0.856131,0.505989,0.189007,-0.138798,-0.068448,-0.462895,-0.105289,5.527365,...,0.765689,0.389700,-0.172694,0.700599,-1.254705,1.445659,1.906148,1.527778,5.732243e+10,6.426946e+10
2,TSLA,2018-12-31,0.831285,0.463829,0.188340,-0.011781,-0.032821,-0.198262,-0.050880,4.871174,...,0.826632,0.818693,-0.003528,0.114225,-17.651361,-0.845082,-0.502350,-0.516484,5.265301e+10,6.093876e+10
3,TSLA,2019-12-31,1.134621,0.711728,0.165555,0.003255,-0.025125,-0.130251,-0.028128,4.055908,...,0.177376,0.006675,-0.120982,-0.071216,0.370818,-1.316406,-0.116886,-0.139860,1.134234e+11,1.205744e+11
4,TSLA,2020-12-31,1.875140,1.492841,0.210236,0.063229,0.013232,0.031046,0.030449,1.308121,...,1.202204,0.629393,0.269889,0.162196,0.857942,23.925000,-2.000000,-2.269589,7.403635e+11,7.327185e+11


symbol                             object
date                               object
currentRatio                      float64
quickRatio                        float64
grossProfitMargin                 float64
operatingProfitMargin             float64
returnOnAssets                    float64
returnOnEquity                    float64
returnOnCapitalEmployed           float64
debtEquityRatio                   float64
priceToBookRatio                  float64
priceToSalesRatio                 float64
priceEarningsRatio                float64
priceEarningsToGrowthRatio        float64
priceToOperatingCashFlowsRatio    float64
growthRevenue                     float64
growthCostOfRevenue               float64
growthGrossProfit                 float64
growthGrossProfitRatio            float64
growthOperatingExpenses           float64
growthEBITDA                      float64
growthOperatingIncome             float64
growthNetIncome                   float64
growthEPS                         

In [2]:
res = api.getFinStat('AAPL', limit=48)
datetime_series = pd.to_datetime(res['date'])
datetime_index = pd.DatetimeIndex(datetime_series.values)
df=res.set_index(datetime_index)
df.drop('date',axis=1,inplace=True)

features = [
    'revenue','costOfRevenue','grossProfit','researchAndDevelopmentExpenses','generalAndAdministrativeExpenses','otherExpenses','operatingExpenses','costAndExpenses','depreciationAndAmortization','ebitda','operatingIncome','totalOtherIncomeExpensesNet','incomeBeforeTax','incomeTaxExpense','netIncome'
]
X = pd.DataFrame({})
for feat in features:
    price = df[feat].squeeze().dropna()
    components = tsa.seasonal_decompose(price, model='additive',period=4)
    X[feat] = components.trend.dropna()

print(X)

pca = PCA(n_components=3)
pca.fit(X)

print(pca.explained_variance_)

                 revenue  costOfRevenue   grossProfit  \
2009-09-26  1.118025e+10   6.843750e+09  4.336500e+09   
2009-12-26  1.276762e+10   7.710250e+09  5.057375e+09   
2010-03-27  1.499712e+10   9.063375e+09  5.933750e+09   
2010-06-26  1.768850e+10   1.078162e+10  6.906875e+09   
2010-09-25  2.046675e+10   1.249988e+10  7.966875e+09   
2010-12-25  2.347162e+10   1.420738e+10  9.264250e+09   
2011-03-26  2.607138e+10   1.560038e+10  1.047100e+10   
2011-06-25  2.951125e+10   1.725612e+10  1.225512e+10   
2011-09-24  3.377512e+10   1.917612e+10  1.459900e+10   
2011-12-31  3.639650e+10   2.037025e+10  1.602625e+10   
2012-03-31  3.816500e+10   2.137712e+10  1.678788e+10   
2012-06-30  4.014938e+10   2.293925e+10  1.721012e+10   
2012-09-29  4.172388e+10   2.474600e+10  1.697788e+10   
2012-12-29  4.231350e+10   2.585875e+10  1.645475e+10   
2013-03-30  4.253925e+10   2.639700e+10  1.614225e+10   
2013-06-29  4.311275e+10   2.693850e+10  1.617425e+10   
2013-09-28  4.375338e+10   2.72